In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install pystan==2.19.1.1
!pip install prophet
from prophet import Prophet

path = "/content/drive/MyDrive/ds_portfolio/kaggle_m5/data"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
df = pd.read_pickle(path + "/02_intermediate/items/FOODS_3_811.pickle")
calendar = pd.read_pickle(path+ "/02_intermediate/calendar_dummy.pickle")

In [5]:
df_subset = df[df["store_id"]=="WI_3"]
start_date = df_subset[df_subset["sell_price"]>0]["date"].reset_index(drop=True)[0]
df_subset = df_subset.query("date >= @start_date")
df_prophet = df_subset.rename({"date": "ds", "num_of_sales": "y"}, axis=1)
df_prophet["floor"] = 0

In [6]:
# Create event dataframe
playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2011-01-08',
                        '2013-01-12',
                        '2014-01-12', 
                        '2014-01-19',
                        '2014-02-02',
                        '2015-01-11', 
                        '2016-01-17',
                        '2016-01-24', 
                        '2016-02-07']),
  'lower_window': 0, # Seemingly affecting on the past
  'upper_window': 1, # Seemingly affecting on the future
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(['2011-02-06',
                        '2012-02-05',
                        '2013-02-03',
                        '2014-02-02', 
                        '2015-02-01',
                        '2016-02-07',]),
  'lower_window': 0,
  'upper_window': 1,
})


# Sum up two events
holidays = pd.concat((playoffs, superbowls))

In [7]:
outside_regs = ['wm_yr_wk', 'wday', 'month', 'year', 'snap_CA',
       'snap_TX', 'snap_WI', 'event_name_1_Chanukah End',
       'event_name_1_Christmas', 'event_name_1_Cinco De Mayo',
       'event_name_1_ColumbusDay', 'event_name_1_Easter',
       'event_name_1_Eid al-Fitr', 'event_name_1_EidAlAdha',
       "event_name_1_Father's day", 'event_name_1_Halloween',
       'event_name_1_IndependenceDay', 'event_name_1_LaborDay',
       'event_name_1_LentStart', 'event_name_1_LentWeek2',
       'event_name_1_MartinLutherKingDay', 'event_name_1_MemorialDay',
       "event_name_1_Mother's day", 'event_name_1_NBAFinalsEnd',
       'event_name_1_NBAFinalsStart', 'event_name_1_NewYear',
       'event_name_1_OrthodoxChristmas', 'event_name_1_OrthodoxEaster',
       'event_name_1_Pesach End', 'event_name_1_PresidentsDay',
       'event_name_1_Purim End', 'event_name_1_Ramadan starts',
       'event_name_1_StPatricksDay', 'event_name_1_SuperBowl',
       'event_name_1_Thanksgiving', 'event_name_1_ValentinesDay',
       'event_name_1_VeteransDay', 'event_type_1_Cultural',
       'event_type_1_National', 'event_type_1_Religious',
       'event_type_1_Sporting', 'event_name_2_Cinco De Mayo',
       'event_name_2_Easter', "event_name_2_Father's day",
       'event_name_2_OrthodoxEaster', 'event_type_2_Cultural',
       'event_type_2_Religious', 'sell_price']

In [8]:
m = Prophet(yearly_seasonality = True, 
            weekly_seasonality = True, 
            daily_seasonality = False,
            changepoint_prior_scale=0.05,
            holidays=holidays,
            seasonality_mode='multiplicative')
m.add_country_holidays(country_name='US')
for reg in outside_regs:
    m.add_regressor(reg, standardize='auto')
m.fit(df_prophet)

In [9]:
df_prophet[["ds", "y"] + outside_regs]

,ds,y,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,event_name_1_Chanukah End,...,event_type_1_National,event_type_1_Religious,event_type_1_Sporting,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster,event_type_2_Cultural,event_type_2_Religious,sell_price
17833,2012-01-28,0,11201,1,1,2012,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.78
17834,2012-01-29,0,11201,2,1,2012,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.78
17835,2012-01-30,0,11201,3,1,2012,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.78
17836,2012-01-31,10,11201,4,1,2012,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.78
17837,2012-02-01,0,11201,5,2,2012,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19405,2016-05-18,21,11616,5,5,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.88
19406,2016-05-19,22,11616,6,5,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.88
19407,2016-05-20,27,11616,7,5,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.88
19408,2016-05-21,27,11617,1,5,2016,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.88


In [10]:
float(df_prophet[df_prophet["ds"]=="2016-05-22"]["sell_price"])

1.88

In [51]:
future = m.make_future_dataframe(periods=28, freq='D')
future = pd.merge(future, calendar, left_on="ds", right_on="date", how="left")
# future[future["ds"]>="2016-05-23"]["sell_price"] = float(df_prophet[df_prophet["ds"]=="2016-05-22"]["sell_price"])
# future.tail(5)
future

ValueError: ignored

In [10]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

ValueError: ignored

In [ ]:
item_name = df.columns[i]

# preprocessing
df_y = df.rename({item_name: "y"}, axis=1)
df_y['floor'] = 0

reggresors = linear_part(price_train, price_predict, df_calendar, item_name)
# 欠損値以外の値が最初に出てくるインデックスを検索(販売開始時点)
start_point = reggresors.reset_index()[reggresors["d"] == reggresors[-pd.isnull(reggresors["price"])].iloc[1, 0]].index[0]

# 販売期間のデータフレーム を作成
df_y = df_y.iloc[start_point-1: , :]
temp = reggresors.iloc[start_point-1: len(df.iloc[start_point-1: , :]) + start_point -1, :].set_index("d")
temp = temp.fillna(reggresors["price"].max() *3)
reggresors_list = temp.columns
df_y = pd.concat([df_y, temp], axis = 1)

# Execute the function
df_y = day_format_converter(df_y.reset_index(), "d", date_df)

# Construct model
if df_y["y"].mean() >= 20:
    model = Prophet(yearly_seasonality = True, 
                    weekly_seasonality = True, 
                    daily_seasonality = False,
                    changepoint_prior_scale=0.05,
                    holidays=holidays,
                    seasonality_mode='multiplicative',
                    )
    model.add_country_holidays(country_name='US')

else:
    model = Prophet(yearly_seasonality = True, 
                    weekly_seasonality = True, 
                    daily_seasonality = False,
                    changepoint_prior_scale=0.05,
                    holidays=holidays,
                    seasonality_mode='additive',
                    )
    model.add_country_holidays(country_name='US')

for j in reggresors_list:
    model.add_regressor(j, standardize='auto')

forecast = model.fit(df_y)

# 予測期間のデータフレームを作成
future = model.make_future_dataframe(periods=28, freq='D')
future = pd.concat([future.set_index("ds"), 
                    day_format_converter(reggresors.iloc[start_point-1: , :], "d", date_df).set_index("ds")], axis = 1)
future = future.reset_index()
future['floor'] = 0
# predict
forecast = model.predict(future)
predict = forecast.tail(56)["yhat"]
predict = predict.reset_index()["yhat"]
predict.name = str(i)
df_forecast = pd.concat([df_forecast, predict], axis = 1)